In [1]:
%run /Users/yfreund@ucsd.edu/Vault

In [2]:
sqlContext

In [3]:
AWS_BUCKET_NAME = "mas-dse-public" 
MOUNT_NAME = "NCDC-weather"
dbutils.fs.unmount("/mnt/%s" % MOUNT_NAME)
output_code=dbutils.fs.mount("s3n://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
#print output_code
file_list=dbutils.fs.ls('/mnt/%s/Spark-Data/mllib/ridge-data/'%MOUNT_NAME)
file_list

In [4]:
from __future__ import print_function

from pyspark.sql import SQLContext
from pyspark.ml.regression import LinearRegression



In [5]:
# $example on$
# Load training data
training = sqlContext.read.format("libsvm")\
    .load("dbfs:/mnt/NCDC-weather/Spark-Data/mllib/sample_linear_regression_data.txt")
# https://github.com/apache/spark/blob/master/data/mllib/sample_linear_regression_data.txt


In [6]:
display(training)

In [7]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for linear regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

### To Do
* Create synthetic 2D datasets: linear model, 2nd order model.
* Split data into train and test
* plot resulting model

In [9]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.replace(',', ' ').split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile("dbfs:/mnt/NCDC-weather/Spark-Data/mllib/ridge-data/lpsa.data")
parsedData = data.map(parsePoint)

# Build the model
model = LinearRegressionWithSGD.train(parsedData)

# Evaluate the model on training data
valuesAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
MSE = valuesAndPreds.map(lambda (v, p): (v - p)**2).reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))

# Save and load model
model.save(sc, "myModelPath")
sameModel = LinearRegressionModel.load(sc, "myModelPath")

In [10]:
display(parsedData.collect())